In [0]:
sc

<SparkContext master=local[8] appName=Databricks Shell>

In [0]:
spark

In [0]:

tickets_2017 = spark.read.csv('s3a://upgrad-spark-data/Parking_Violations_Issued_-_Fiscal_Year_2017.csv',inferSchema=True,header=True)
tickets_2017.show(1)

+--------------+--------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+-------------------+----------------+---------------------------------+------------+--------------------+-------------------+-------------------+-----------+------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------+------------+--------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+
|Summons Number|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicle Expiration Date|Violation Location|Violation Precinct|Issuer Precinct|Is

In [0]:
tickets_2017.columns
tickets_2017.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: date (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation County

In [0]:
# Examine the data

# Find the total number of tickets for the year.
# Find out the number of unique states from where the cars that got parking tickets came. (Hint: Use the column 'Registration State'.)
# There is a numeric entry '99' in the column, which should be corrected. Replace it with the state having the maximum entries. Provide the number of unique states again.

In [0]:
tickets_2017.count()

10803028

In [0]:
# #tickets_2017.select('Registration State').groupBy('Registration State').count().orderBy('count', ascending=False).show(truncate=False)
from pyspark.sql.functions import *
from pyspark.sql import *

td=tickets_2017.withColumn('Registration State',when(tickets_2017['Registration State']=='99','NY').otherwise(tickets_2017['Registration State']))

In [0]:
td.select('Registration State').groupBy('Registration State').count().orderBy('count', ascending=False).show(1000,truncate=False)


+------------------+-------+
|Registration State|count  |
+------------------+-------+
|NY                |8517686|
|NJ                |925965 |
|PA                |285419 |
|FL                |144556 |
|CT                |141088 |
|MA                |85547  |
|IN                |80749  |
|VA                |72626  |
|MD                |61800  |
|NC                |55806  |
|IL                |37329  |
|GA                |36852  |
|TX                |36516  |
|AZ                |26426  |
|OH                |25302  |
|CA                |24260  |
|SC                |21836  |
|ME                |21574  |
|MN                |18227  |
|OK                |18165  |
|TN                |17275  |
|DE                |16325  |
|MI                |15703  |
|RI                |12224  |
|NH                |8752   |
|VT                |7367   |
|AL                |6891   |
|WA                |6311   |
|ON                |5601   |
|OR                |5483   |
|MO                |4454   |
|QB           

In [0]:
td.select('Registration State').distinct().show(68,truncate=False)

+------------------+
|Registration State|
+------------------+
|AZ                |
|SC                |
|NS                |
|LA                |
|MN                |
|NJ                |
|DC                |
|OR                |
|NT                |
|VA                |
|RI                |
|WY                |
|KY                |
|BC                |
|NH                |
|MI                |
|GV                |
|NV                |
|QB                |
|WI                |
|ID                |
|CA                |
|CT                |
|NE                |
|MT                |
|NC                |
|VT                |
|MD                |
|DE                |
|MO                |
|IL                |
|ME                |
|MB                |
|WA                |
|ND                |
|MS                |
|AL                |
|IN                |
|OH                |
|TN                |
|IA                |
|NM                |
|PA                |
|SD                |
|FO          

In [0]:
# Aggregation tasks

# How often does each violation code occur? Display the frequency of the top five violation codes.
# How often does each 'vehicle body type' get a parking ticket? How about the 'vehicle make'? (Hint: Find the top 5 for both.)

In [0]:
spark.sql('select `violation code`,count(*) as total from {td} group by `violation code` order by total desc limit 5',td=td).show()

+--------------+-------+
|violation code|  total|
+--------------+-------+
|            21|1528588|
|            36|1400614|
|            38|1062304|
|            14| 893498|
|            20| 618593|
+--------------+-------+



In [0]:
spark.sql('select `vehicle body type`,count(*) as total from {td} group by `vehicle body type` order by total desc limit 5',td=td).show(truncate=False)

+-----------------+-------+
|vehicle body type|total  |
+-----------------+-------+
|SUBN             |3719802|
|4DSD             |3082020|
|VAN              |1411970|
|DELV             |687330 |
|SDN              |438191 |
+-----------------+-------+



In [0]:
spark.sql('select `vehicle make`,count(*) as total from {td} group by `vehicle make` order by total desc limit 5',td=td).show(truncate=False)

+------------+-------+
|vehicle make|total  |
+------------+-------+
|FORD        |1280958|
|TOYOT       |1211451|
|HONDA       |1079238|
|NISSA       |918590 |
|CHEVR       |714655 |
+------------+-------+



In [0]:
#3
spark.sql('select `violation precinct`,count(*) as total from {td} group by `violation precinct` order by total desc limit 6',td=td).show(truncate=False)

+------------------+-------+
|violation precinct|total  |
+------------------+-------+
|0                 |2072400|
|19                |535671 |
|14                |352450 |
|1                 |331810 |
|18                |306920 |
|114               |296514 |
+------------------+-------+



In [0]:
spark.sql('select `issuer precinct`,count(*) as total from {td} group by `issuer precinct` order by total desc limit 6',td=td).show(truncate=False)

+---------------+-------+
|issuer precinct|total  |
+---------------+-------+
|0              |2388479|
|19             |521513 |
|14             |344977 |
|1              |321170 |
|18             |296553 |
|114            |289950 |
+---------------+-------+



In [0]:
#4
# Find the violation code frequencies for three precincts that have issued the most number of tickets. Do these precinct zones have an exceptionally high frequency of certain violation codes? Are these codes common across precincts? 
#(Hint: In the SQL view, use the 'where' attribute to filter among three precincts.)

spark.sql('select `Issuer Precinct`,`violation code`,count(*) as total from {td} where `violation precinct`=19 group by `violation code`,`Issuer Precinct` order by total desc limit 5',td=td).show(truncate=False)

+---------------+--------------+-----+
|Issuer Precinct|violation code|total|
+---------------+--------------+-----+
|19             |46            |86197|
|19             |37            |72437|
|19             |38            |72341|
|19             |14            |57523|
|19             |21            |54699|
+---------------+--------------+-----+



In [0]:
spark.sql('select `Issuer Precinct`,`violation code`,count(*) as total from {td} where `violation precinct`=14 group by `violation code`,`Issuer Precinct` order by total desc limit 5',td=td).show(truncate=False)

+---------------+--------------+-----+
|Issuer Precinct|violation code|total|
+---------------+--------------+-----+
|14             |14            |73703|
|14             |69            |58026|
|14             |31            |39851|
|14             |47            |30534|
|14             |42            |20663|
+---------------+--------------+-----+



In [0]:
spark.sql('select `Issuer Precinct`,`violation code`,count(*) as total from {td} where `violation precinct`=1 group by `violation code`,`Issuer Precinct` order by total desc limit 5',td=td).show(truncate=False)

+---------------+--------------+-----+
|Issuer Precinct|violation code|total|
+---------------+--------------+-----+
|1              |14            |72766|
|1              |16            |38915|
|1              |20            |26722|
|1              |46            |22435|
|1              |38            |16981|
+---------------+--------------+-----+



In [0]:
# Find out the properties of parking violations across different times of the day:
# Find a way to deal with missing values, if any.
# (Hint: Check for the null values using 'isNull' under the SQL. Also, to remove the null values, check the 'dropna' command in the API documentation.)

# The Violation Time field is specified in a strange format. Find a way to make this a time attribute that you can use to divide into groups.

# Divide 24 hours into six equal discrete bins of time. Choose the intervals as you see fit. For each of these groups, find the three most commonly occurring violations.
# (Hint: Use the CASE-WHEN in SQL view to segregate into bins. To find the most commonly occurring violations, you can use an approach similar to the one mentioned in the hint for question 4.)

# Now, try another direction. For the three most commonly occurring violation codes, find the most common time of the day (in terms of the bins from the previous part).


spark.sql("select count(*) as count\
                 FROM {td} where `Violation Time` is Null",td=td).show()

+-----+
|count|
+-----+
|   63|
+-----+



In [0]:
help(td.dropna)

Help on method dropna in module pyspark.sql.dataframe:

dropna(how: str = 'any', thresh: Optional[int] = None, subset: Union[str, Tuple[str, ...], List[str], NoneType] = None) -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` omitting rows with null values.
    :func:`DataFrame.dropna` and :func:`DataFrameNaFunctions.drop` are
    aliases of each other.
    
    .. versionadded:: 1.3.1
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    how : str, optional, the values that can be 'any' or 'all', default 'any'.
        If 'any', drop a row if it contains any nulls.
        If 'all', drop a row only if all its values are null.
    thresh: int, optional, default None.
        If specified, drop rows that have less than `thresh` non-null values.
        This overwrites the `how` parameter.
    subset : str, tuple or list, optional
        optional list of column names to consider.
   

In [0]:
null_values=td.dropna(subset='Violation Time')
null_values.count()

10802965

In [0]:
spark.sql("select count(*) as count\
                 FROM {null_values} where `Violation Time` is Null",null_values=null_values).show()

+-----+
|count|
+-----+
|    0|
+-----+



In [0]:
 data=null_values

In [0]:
data.count()

10802965

In [0]:
data1=spark.sql("SELECT `Violation code`,`Violation Time`\
    ,CASE WHEN substring(`violation time`,-1) = 'A' THEN concat(substring(`violation time`,1,2),':',substring(`violation time`,3,3),'M') \
        ELSE concat(int(substring(`violation time`,1,2))+12,':',substring(`violation time`,3,3),'M')\
             END as Violation_Time FROM {data}",data=data)

data1.show(10)

+--------------+--------------+--------------+
|Violation code|Violation Time|Violation_Time|
+--------------+--------------+--------------+
|             7|         0143A|       01:43AM|
|             7|         0400P|       16:00PM|
|             5|         0233P|       14:33PM|
|            47|         1120A|       11:20AM|
|            69|         0555P|       17:55PM|
|             7|         0852P|       20:52PM|
|            40|         0215A|       02:15AM|
|            36|         0758A|       07:58AM|
|            36|         1005A|       10:05AM|
|             5|         0845A|       08:45AM|
+--------------+--------------+--------------+
only showing top 10 rows



In [0]:
data2=spark.sql('''select `Violation_Time`,`Violation Code`,case WHEN  (int( substring (`Violation_time`,1,2))) between 00 and 04 then 'Late_night' 
                         WHEN  (int( substring (`Violation_time`,1,2))) between 05 and 08 then 'Early_Morning' 
                         WHEN  (int( substring (`Violation_time`,1,2))) between 09 and 12 then 'Morning' 
                         WHEN  (int( substring (`Violation_time`,1,2))) between 13 and 16 then 'Afternoon' 
                         WHEN  (int( substring (`Violation_time`,1,2))) between 17 and 20 then 'Evening' 
                         Else 'Night' 
                         end as bins from {data1}''',data1=data1)
data2.show(10)

+--------------+--------------+-------------+
|Violation_Time|Violation Code|         bins|
+--------------+--------------+-------------+
|       01:43AM|             7|   Late_night|
|       16:00PM|             7|    Afternoon|
|       14:33PM|             5|    Afternoon|
|       11:20AM|            47|      Morning|
|       17:55PM|            69|      Evening|
|       20:52PM|             7|      Evening|
|       02:15AM|            40|   Late_night|
|       07:58AM|            36|Early_Morning|
|       10:05AM|            36|      Morning|
|       08:45AM|             5|Early_Morning|
+--------------+--------------+-------------+
only showing top 10 rows



In [0]:
spark.sql('select bins,`violation code`,count(*) as total from {data2} where bins="Night" group by bins,`violation code` order by total desc',data2=data2).show(10)

+-----+--------------+------+
| bins|violation code| total|
+-----+--------------+------+
|Night|            36|211434|
|Night|            21|144004|
|Night|            38|135350|
|Night|            14| 88122|
|Night|            37| 77726|
|Night|             7| 77714|
|Night|            46| 73428|
|Night|            20| 64529|
|Night|            40| 63603|
|Night|            71| 52246|
+-----+--------------+------+
only showing top 10 rows



In [0]:
spark.sql('select bins,`violation code`,count(*) as total from {data2} where bins="Late_night" group by bins,`violation code` order by total desc',data2=data2).show(10)

+----------+--------------+-----+
|      bins|violation code|total|
+----------+--------------+-----+
|Late_night|            21|74996|
|Late_night|            40|52000|
|Late_night|            78|31628|
|Late_night|            14|31530|
|Late_night|            20|24986|
|Late_night|             7|20412|
|Late_night|            85|13534|
|Late_night|            46|12405|
|Late_night|            19|11173|
|Late_night|            71| 8701|
+----------+--------------+-----+
only showing top 10 rows



In [0]:
spark.sql('select bins,`violation code`,count(*) as total from {data2} where bins="Early_Morning" group by bins,`violation code` order by total desc',data2=data2).show(10)

+-------------+--------------+------+
|         bins|violation code| total|
+-------------+--------------+------+
|Early_Morning|            21|498338|
|Early_Morning|            14|207958|
|Early_Morning|            36|170484|
|Early_Morning|            40|139382|
|Early_Morning|            20|129826|
|Early_Morning|            71| 85693|
|Early_Morning|             7| 62859|
|Early_Morning|            46| 57440|
|Early_Morning|            38| 52544|
|Early_Morning|            19| 47441|
+-------------+--------------+------+
only showing top 10 rows



In [0]:
spark.sql('select bins,`violation code`,count(*) as total from {data2} where bins="Morning" group by bins,`violation code` order by total desc',data2=data2).show(10)

+-------+--------------+------+
|   bins|violation code| total|
+-------+--------------+------+
|Morning|            21|806290|
|Morning|            36|614877|
|Morning|            38|296251|
|Morning|            14|208171|
|Morning|            46|190134|
|Morning|            71|149799|
|Morning|            20|130159|
|Morning|            40|104270|
|Morning|            37| 98952|
|Morning|             7| 91230|
+-------+--------------+------+
only showing top 10 rows



In [0]:
spark.sql('select bins,`violation code`,count(*) as total from {data2} where bins="Afternoon" group by bins,`violation code` order by total desc',data2=data2).show(10)

+---------+--------------+------+
|     bins|violation code| total|
+---------+--------------+------+
|Afternoon|            38|453644|
|Afternoon|            36|394403|
|Afternoon|            37|339739|
|Afternoon|            14|279225|
|Afternoon|            20|215705|
|Afternoon|            46|209327|
|Afternoon|            71|190989|
|Afternoon|             7|151209|
|Afternoon|            40|123009|
|Afternoon|            70| 99882|
+---------+--------------+------+
only showing top 10 rows



In [0]:
spark.sql('select bins,`violation code`,count(*) as total from {data2} where bins="Evening" group by bins,`violation code` order by total desc',data2=data2).show(10)

+-------+--------------+------+
|   bins|violation code| total|
+-------+--------------+------+
|Evening|            38|124026|
|Evening|             7|112971|
|Evening|            14| 78491|
|Evening|            37| 77800|
|Evening|            46| 57278|
|Evening|            20| 53388|
|Evening|             5| 45151|
|Evening|            40| 37349|
|Evening|            71| 33880|
|Evening|            19| 26310|
+-------+--------------+------+
only showing top 10 rows

